# Analysing Startup Fundraising Deals from Crunchbase


## Introduction

In this project, we will use Pandas along with memory optimisation techniques, chunk data processing and SQLite integration to allow us to work with larger datasets.
Specifically, we'll analyse startup investments from [Crunchbase.com](https://www.crunchbase.com/), which is a website that crowdsources information on the fundraising rounds of many startups.

The [dataset of investments](https://github.com/datahoarder/crunchbase-october-2013/blob/master/crunchbase-investments.csv) we'll be exploring is from October 2013. Each row is an individual investment from a venture capitalist or institution - they contain information such as the name of the startup and investor alongside their locations, as well as the amount and date of the investment itself.

Throughout this project, we'll practice working with different memory constraints. For context, the `crunchbase-investments.csv` consumes 10.3 MB of disk space.

In this first step, we'll explore our data to get an idea of how we can optimise the datatypes and how we might query our dataset.
We'll assume we only have 10 MB of available memory. This means we need our chunks to consume less than 50% of our available memory.

Since the dataset contains over 50,000 rows, let's read the dataset into dataframes using 5,000 row chunks.


In [1]:
# How many missing values are in each column?

import pandas as pd
import numpy as np

crunchbase_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000)

missing = []
for chunk in crunchbase_iter:
    missing_values = chunk.isnull().sum(axis=0)
    missing.append(missing_values)

missing_combined = pd.concat(missing)
missing_final = missing_combined.groupby(missing_combined.index, axis=0).sum().sort_values()
print(missing_final)
        
    

company_country_code          1
company_name                  1
company_permalink             1
company_region                1
investor_region               2
investor_permalink            2
investor_name                 2
funded_quarter                3
funded_at                     3
funded_month                  3
funded_year                   3
funding_round_type            3
company_state_code          492
company_city                533
company_category_code       643
raised_amount_usd          3599
investor_country_code     12001
investor_city             12480
investor_state_code       16809
investor_category_code    50427
dtype: int64


In [2]:
# How much memory does each column consume (MB)?

crunchbase_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000)

col_memory = {}
for chunk in crunchbase_iter:
    for col in chunk.columns:
        memory = chunk[col].memory_usage(deep=True) / 2**20
        if col in col_memory:
            col_memory[col] += memory
        else:
            col_memory[col] = memory
            
for col in col_memory:
    print(col, ': ', col_memory[col])
    print('\n')
    

company_permalink :  3.8711891174316406


company_name :  3.4269304275512695


company_category_code :  3.2639999389648438


company_country_code :  3.0266036987304688


company_state_code :  2.9635419845581055


company_region :  3.2548837661743164


company_city :  3.3448543548583984


investor_permalink :  4.751201629638672


investor_name :  3.737185478210449


investor_category_code :  0.594970703125


investor_country_code :  2.5260353088378906


investor_state_code :  2.36325740814209


investor_region :  3.2404909133911133


investor_city :  2.7536821365356445


funding_round_type :  3.254084587097168


funded_at :  3.379471778869629


funded_month :  3.2282180786132812


funded_quarter :  3.2282180786132812


funded_year :  0.40474700927734375


raised_amount_usd :  0.40474700927734375




In [3]:
# What is the total memory footprint of all the chunks combined (MB)?

total_memory = 0
for col in col_memory:
    total_memory += col_memory[col]

print(total_memory)

57.01831340789795


In [4]:
# Which columns are not useful for analysis purposes and can be dropped?

crunchbase_first_rows = pd.read_csv('crunchbase-investments.csv', nrows=5)

print(crunchbase_first_rows)

     company_permalink company_name company_category_code  \
0    /company/advercar     AdverCar           advertising   
1  /company/launchgram   LaunchGram                  news   
2        /company/utap         uTaP             messaging   
3    /company/zoopshop     ZoopShop              software   
4    /company/efuneral     eFuneral                   web   

  company_country_code company_state_code         company_region  \
0                  USA                 CA                 SF Bay   
1                  USA                 CA                 SF Bay   
2                  USA                NaN  United States - Other   
3                  USA                 OH               Columbus   
4                  USA                 OH              Cleveland   

    company_city          investor_permalink      investor_name  \
0  San Francisco  /company/1-800-flowers-com  1-800-FLOWERS.COM   
1  Mountain View        /company/10xelerator        10Xelerator   
2            NaN       

Looking at the contents of the columns above, we can see that the `company_permalink` and `investor_permalink` columns are not useful as we already have the names of the companies/investors in other columns.
Also, the `funded_month`, `funded_quarter` and `funded_year` columns are not useful as this information can be obtained using the `funded_at` column.

Let's drop the columns that we do not need.

In [5]:
# Create column list with only the useful columns. This will be used when reading in the csv moving forward.

cols_to_drop = ['company_permalink', 'investor_permalink', 'funded_month', 'funded_quarter', 'funded_year']

useful_cols = []
for col in crunchbase_first_rows.columns:
    if col not in cols_to_drop:
        useful_cols.append(col)

print(useful_cols)

['company_name', 'company_category_code', 'company_country_code', 'company_state_code', 'company_region', 'company_city', 'investor_name', 'investor_category_code', 'investor_country_code', 'investor_state_code', 'investor_region', 'investor_city', 'funding_round_type', 'funded_at', 'raised_amount_usd']


## Selecting Data Types

Now that we have an idea of the missing values in each column and their memory footprints, we can take a look at their datatypes.
This will allow us to optimise our dataframe by selecting the most appropriate datatypes. We'll aim to reduce the overall memory the data consumes to under 10 megabytes.

In [6]:
# Let's print the type of each column now we have dropped the ones that are not useful for analysis. We shall also look to see if the types shift between chunks.

crunchbase_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, usecols=useful_cols)

col_dtypes = {}
for chunk in crunchbase_iter:
    for col in chunk.columns:
        col_dtype = str(chunk[col].dtype)
        if col in col_dtypes:
            if col_dtype not in col_dtypes[col]:
                col_dtypes[col].append(col_dtype)
        else:
            col_dtypes[col] = [col_dtype]

for col in col_dtypes:
    print(col, ':', col_dtypes[col])
    print('\n')
    

company_name : ['object']


company_category_code : ['object']


company_country_code : ['object']


company_state_code : ['object']


company_region : ['object']


company_city : ['object']


investor_name : ['object']


investor_category_code : ['object', 'float64']


investor_country_code : ['object', 'float64']


investor_state_code : ['object', 'float64']


investor_region : ['object']


investor_city : ['object', 'float64']


funding_round_type : ['object']


funded_at : ['object']


raised_amount_usd : ['float64']




As the `investor_category_code`, `investor_country_code`, `investor_state_code` and `investor_city` columns shift between types, we will explicitly read them as object types moving forward.

We can also parse the `funded_at` column as a datetime moving forward as this is a more memory efficient way to represent dates.

The only remaining numeric column will then be `raised_amount_usd`. As float types can lose accuracy with numbers higher than a certain value, we will keep this column as a float64 to prevent that from happening.

Now let's analyse the unique value counts across all of the chunks to see if we can convert them to a category type.

In [7]:
# Do any of the columns have less than 50% unique values and are therefore candidates for conversion into the category type?

dtypes = {'investor_category_code': 'object', 'investor_country_code': 'object', 'investor_state_code': 'object', 'investor_city': 'object'}

crunchbase_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, usecols=useful_cols, parse_dates=['funded_at'], dtype=dtypes) # Parse funded_at as a datetime

column_vc = {}
for chunk in crunchbase_iter:
    string_cols = chunk.select_dtypes(include='object')
    for col in string_cols.columns:
        value_counts = string_cols[col].value_counts()
        if col in column_vc:
            column_vc[col].append(value_counts)
        else:
            column_vc[col] = [value_counts]

combined_vc = {}
for col in column_vc:
    combined_vc[col] = pd.concat(column_vc[col], axis=0)
    combined_vc[col] = combined_vc[col].groupby(combined_vc[col].index).sum()

# Count the unique and total values per column. Then print the columns with less than 50% unique
unique_values = {}
total_values = {}
convert_to_category = []
for col in combined_vc:
    unique_count = len(combined_vc[col])
    unique_values[col] = unique_count
    
    total_count = combined_vc[col].sum()
    total_values[col] = total_count

    percentage_unique = (unique_values[col] / total_values[col]) * 100
    if percentage_unique < 50:
        convert_to_category.append(col)
        
print(convert_to_category)

['company_name', 'company_category_code', 'company_country_code', 'company_state_code', 'company_region', 'company_city', 'investor_name', 'investor_category_code', 'investor_country_code', 'investor_state_code', 'investor_region', 'investor_city', 'funding_round_type']


Now let's read the columns identified as having less than 50% unique values into our dataframe chunks as the category type.
We can also print the overall memory footprint of our data to if we have reduced it below our 10 megabyte target.

In [8]:
# Update dtypes dictionary to include the above columns as category types
for col in convert_to_category:
    if col in dtypes:
        dtypes[col] = 'category'
    else:
        dtypes[col] = 'category'

crunchbase_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, usecols=useful_cols, parse_dates=['funded_at'], dtype=dtypes) 

# What's the total memory consumption of all chunks combined (MB)?
total_memory_usage = 0
for chunk in crunchbase_iter:
    total_memory_usage += chunk.memory_usage(deep=True).sum()

print(total_memory_usage / 2**20)


6.601466178894043


## Loading Chunks into SQLite

Now we're in good shape to start exploring and analysing the data. Our next step is to load each chunk into a SQLite database so we can query the full dataset.

In [9]:
# Load the chunks into an SQLite database table

import sqlite3

conn = sqlite3.connect('crunchbase.db')

crunchbase_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, usecols=useful_cols, parse_dates=['funded_at'], dtype=dtypes) 

for chunk in crunchbase_iter:
    chunk.to_sql('crunchbase', conn, if_exists='append', index=False)
    
# Query table and make sure the datatypes match what we expect

table_types = pd.read_sql('PRAGMA table_info(crunchbase)', conn)
print(table_types)

    cid                    name       type  notnull dflt_value  pk
0     0            company_name       TEXT        0       None   0
1     1   company_category_code       TEXT        0       None   0
2     2    company_country_code       TEXT        0       None   0
3     3      company_state_code       TEXT        0       None   0
4     4          company_region       TEXT        0       None   0
5     5            company_city       TEXT        0       None   0
6     6           investor_name       TEXT        0       None   0
7     7  investor_category_code       TEXT        0       None   0
8     8   investor_country_code       TEXT        0       None   0
9     9     investor_state_code       TEXT        0       None   0
10   10         investor_region       TEXT        0       None   0
11   11           investor_city       TEXT        0       None   0
12   12      funding_round_type       TEXT        0       None   0
13   13               funded_at  TIMESTAMP        0       None

## Use the Pandas SQLite Workflow to Answer Questions

Now that the data is in SQLite, we can use the Pandas SQLite workflow to explore and analyse the startup investments in the Crunchbase dataset. We are able to read in subsets of the data using SQL to ensure it fits into the available memory and then use Pandas to run the calculations.
As our entire dataset now only consumes 6.6 MB in memory (below our 10 MB target), we don't have to worry about our SQL results being too large.

We will answer the following questions:

* What proportion of the total amount of funds did the top 10% of startups raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.
* Which category of company attracted the most investment?
* Which investor contributed the most money (across all startups)?
* Which investors contributed the most money per startup?
* Which funding round was the most popular? Which was the least popular?


In [10]:
# What proportion of the total amount of funds did the top 10% and 1% raise? What about the bottom 10% and 1%?

pd.set_option('display.float_format', lambda x: '%.3f' % x) # Suppress scientific notation

crunchbase_data = pd.read_sql(
                              '''
                              SELECT company_name,
                                     SUM(raised_amount_usd) as total_raised
                                FROM crunchbase
                               GROUP BY company_name
                               ORDER BY total_raised DESC;
                              ''',
                              conn
                             )

crunchbase_data = crunchbase_data.loc[crunchbase_data['total_raised'].notnull(), :] # Remove companies that have missing values in the total_raised column

company_count = crunchbase_data['company_name'].count()
overall_funds = crunchbase_data['total_raised'].sum()

top_10_percent_companies = crunchbase_data.head((company_count // 100) * 10)
top_1_percent_companies = crunchbase_data.head((company_count // 100) * 1)
bottom_10_percent_companies = crunchbase_data.tail((company_count // 100) * 10)
bottom_1_percent_companies = crunchbase_data.tail((company_count // 100) * 1)

top_10_proportion = (top_10_percent_companies['total_raised'].sum() / overall_funds) * 100
top_1_proportion = (top_1_percent_companies['total_raised'].sum() / overall_funds) * 100
bottom_10_proportion = (bottom_10_percent_companies['total_raised'].sum() / overall_funds) * 100
bottom_1_proportion = (bottom_1_percent_companies['total_raised'].sum() / overall_funds) * 100

print('Top 10% of companies:', top_10_proportion)
print('Top 1% of companies:', top_1_proportion)
print('Bottom 10% of companies:', bottom_10_proportion)
print('Bottom 1% of companies:', bottom_1_proportion)

Top 10% of companies: 64.42804146771424
Top 1% of companies: 25.040017618584383
Bottom 10% of companies: 0.026323288553174094
Bottom 1% of companies: 0.0002172554011867574


In [11]:
# Which category of company attracted the most investment?

crunchbase_data = pd.read_sql(
                              '''
                              SELECT company_category_code,
                                     SUM(raised_amount_usd) as total_raised
                                FROM crunchbase
                               GROUP BY company_category_code;
                              ''',
                              conn
                             )

crunchbase_data.sort_values('total_raised', axis=0, inplace=True, ascending=False)

print(crunchbase_data)

   company_category_code     total_raised
5                biotech 441585692248.000
39              software 292338066896.000
24                mobile 259109519008.000
6              cleantech 210820900112.000
11            enterprise 183443709092.000
43                   web 160573059956.000
22               medical 101468421124.000
2            advertising 100306647516.000
9              ecommerce  90268880284.000
27       network_hosting  89678735360.000
37         semiconductor  89433348932.000
16              hardware  83781803676.000
14           games_video  77638594776.000
3              analytics  54976938116.000
36              security  52599484116.000
38                social  46121721352.000
13               finance  44800810204.000
17                health  35157194004.000
26              nanotech  33833280000.000
33      public_relations  31260936000.000
4             automotive  28957006320.000
30                 other  25065763220.000
35                search  23391159

In [12]:
# Which investor contributed the most money across all startups?

crunchbase_data = pd.read_sql(
                              '''
                              SELECT investor_name,
                                     SUM(raised_amount_usd) as total_raised
                                FROM crunchbase
                               GROUP BY investor_name;
                              ''',
                              conn
                             )

biggest_investor_overall = crunchbase_data.sort_values('total_raised', ascending=False, axis=0).head(1)
print(biggest_investor_overall)

                         investor_name    total_raised
5375  Kleiner Perkins Caufield & Byers 44871305504.000


In [13]:
# Which investors contributed the most money per startup?

crunchbase_data = pd.read_sql(
                              '''
                              SELECT company_name,
                                     investor_name,
                                     MAX(raised_amount_usd) as total_contributed
                                FROM crunchbase
                               GROUP BY company_name
                               ORDER BY total_contributed DESC ;
                              ''',
                              conn
                             )

crunchbase_data = crunchbase_data.loc[crunchbase_data['total_contributed'].notnull(), :]

print(crunchbase_data)


             company_name                                      investor_name  \
0               Clearwire                                        BrightHouse   
1               sigmacare                             Marlin Equity Partners   
2                Facebook                           Digital Sky Technologies   
3          Wave Broadband                                        GI Partners   
4                     AOL                                             Google   
...                   ...                                                ...   
10352          WhiteWilly                                         john bialk   
10353  PictureMe Universe  UW-Eau Claire Entrepreneur Program CEO Idea Ch...   
10354            IndyGeek                                           John Fox   
10355            uromovie                                                cnc   
10356   Main Street Stark                                    Jeff SKI Kinsey   

       total_contributed  
0         32

In [14]:
# Which funding round was the most popular? Which was the least popular?

crunchbase_data = pd.read_sql('SELECT company_name, investor_name AS investors FROM crunchbase;', conn)

company_investment_count = crunchbase_data.groupby('company_name', axis=0).count()

most_popular = company_investment_count.loc[company_investment_count['investors'] == company_investment_count['investors'].max(), :]
least_popular = company_investment_count.loc[company_investment_count['investors'] == company_investment_count['investors'].min(), :]

print(most_popular)
print(least_popular)

              investors
company_name           
ecomom              232
              investors
company_name           
series-c+             0
